# Two step CMR search


First we use the collection search API to find collections of interest.

    GET https://cmr.earthdata.nasa.gov/search/collections

Say, for example, we are interested in surface reflectance data from the Sentinel-2 satellite over New York City on June 26th, 2021.

Let's translate the above into a set of query filters.

## Spatial
New York City is located at 40.70125° North, 73.98581° West. We can use the `point` search filter to express this

    point=-73.98581,40.70125

## Temporal 
June 26th, 2021 can be described using an ISO8601 format as `2021-06-21T00:00:00Z,2021-06-21T23:59:59Z`. We can use the `temporal` search filter to express this

    temporal[]=2021-06-21T00:00:00Z,2021-06-21T23:59:59Z

## Satellite and measurement
Finally, our satellite and measurement can be expressed using a keyword (free text) search filter.

    keyword=sentinel-2 surface reflectance

## The collection search

We can AND all these terms together and form a CMR query as follows:

    GET https://cmr.earthdata.nasa.gov/search/collections?keyword=sentinel-2 surface reflectance&point=-73.98581,40.70125&temporal[]=2021-06-21T00:00:00Z,2021-06-21T23:59:59Z


In [66]:
import requests
import xml.dom.minidom
import xml.etree.ElementTree as et

response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?keyword=sentinel-2 surface reflectance&point=-73.98581,40.70125&temporal[]=2021-06-21T00:00:00Z,2021-06-21T23:59:59Z")
print("Number of matching collections: " + response.headers['CMR-Hits'])

Number of matching collections: 4


We have a number of results. Let's look at the first one:

In [67]:
doc = et.fromstring(response.text)

references = doc.findall('references/reference')
reference = references[0].find('location').text
concept_id = references[0].find('id').text
print("First collection reference: " + str(reference))
print("First collection name: " + str(references[0].find('name').text))
print("First collection concept id: " + str(concept_id))

First collection reference: https://cmr.earthdata.nasa.gov:443/search/concepts/C1711924822-LPCLOUD/42
First collection name: HLS Sentinel-2 Multi-spectral Instrument Surface Reflectance Daily Global 30 m V1.5
First collection concept id: C1711924822-LPCLOUD


That looks good. Let's look at the actual metadata associated with that result.
## The collection retrieval

In [68]:
response = requests.get(reference + ".echo10")
response_as_dom = xml.dom.minidom.parseString(response.text)
xml_reference_response = response_as_dom.toprettyxml()
print(xml_reference_response)

<?xml version="1.0" ?>
<Collection>
	<ShortName>HLSS30</ShortName>
	<VersionId>1.5</VersionId>
	<InsertTime>2014-04-03T00:00:00.000Z</InsertTime>
	<LastUpdate>2020-03-04T07:19:53.396Z</LastUpdate>
	<LongName>Not provided</LongName>
	<DataSetId>HLS Sentinel-2 Multi-spectral Instrument Surface Reflectance Daily Global 30 m V1.5</DataSetId>
	<Description>PROVISIONAL - The Harmonized Landsat and Sentinel-2 (HLS)  data have not been validated for their science quality and should not be used in science research or applications. The HLS project provides consistent surface reflectance data from the Operational Land Imager (OLI) aboard the joint NASA/USGS Landsat 8 satellite and the Multi-Spectral Instrument (MSI) aboard the European Union’s Copernicus Sentinel-2A and Sentinel-2B satellites. The combined measurement enables global observations of the land every 2-3 days at 30 meter (m) spatial resolution. The HLS project uses a set of algorithms to obtain seamless products from OLI and MSI that

This looks like what we want. What else can we do at the collection level to assess whether we have the right collection?
It has a browse image. Let's look at that. Note: this is a browse image for the whole collection. It's just an example
## Extract the collection browse image and visualize

In [69]:
doc = et.fromstring(response.text)
browse_url = doc.findall("AssociatedBrowseImageUrls/ProviderBrowseUrl/URL")[0].text
print("Collection browse URL: " + str(browse_url)) 

from PIL import Image

img = Image.open(requests.get(browse_url, stream=True).raw)
img. show()

Collection browse URL: https://cmr.earthdata.nasa.gov/browse-scaler/browse_images/granules/G1947786851-LPCLOUD?h=512&w=512


## Searching for granules
Assuming we are happy with this collection as the source of our data, we can now use the concept-id of that collection to search for granules.
We will use the spatial and temporal filters we used in our collection search. We no longer need the keyword filter

    https://cmr.earthdata.nasa.gov/search/granules.echo10?collection_concept_id=foo&point=-73.98581,40.70125&temporal[]=2021-06-21T00:00:00Z,2021-06-21T23:59:59Z

In [70]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/granules.echo10?collection_concept_id=" + concept_id + "&point=-73.98581,40.70125&temporal[]=2021-06-21T00:00:00Z,2021-06-21T23:59:59Z")
print("Number of matching granules: " + response.headers['CMR-Hits'])


Number of matching granules: 1


That search matched a single granule.

## Inspecting the granule metadata
Let's look at the metadata for this granule

In [71]:
response_as_dom = xml.dom.minidom.parseString(response.text)
xml_reference_response = response_as_dom.toprettyxml()
print(xml_reference_response)


<?xml version="1.0" ?>
<results>
	<hits>1</hits>
	<took>294</took>
	<result collection-concept-id="C1711924822-LPCLOUD" concept-id="G2079468935-LPCLOUD" format="application/echo10+xml" revision-id="1">
		

		<Granule>
			
  
			<GranuleUR>HLS.S30.T18TWL.2021172T153911.v1.5</GranuleUR>
			
  
			<InsertTime>2021-06-25T19:55:22.628524Z</InsertTime>
			
  
			<LastUpdate>2021-06-25T19:55:22.628531Z</LastUpdate>
			
  
			<Collection>
				
    
				<DataSetId>HLS Sentinel-2 Multi-spectral Instrument Surface Reflectance Daily Global 30 m V1.5</DataSetId>
				
  
			</Collection>
			
  
			<DataGranule>
				
    
				<DataGranuleSizeInBytes>203720254</DataGranuleSizeInBytes>
				
    
				<ProducerGranuleId>HLS.S30.T18TWL.2021172T153911</ProducerGranuleId>
				
    
				<DayNightFlag>DAY</DayNightFlag>
				
    
				<ProductionDateTime>2021-06-25T19:54:23.000000Z</ProductionDateTime>
				
    
				<LocalVersionId>1.5</LocalVersionId>
				
  
			</DataGranule>
			
  
			<Temporal>
				
    
	

## Extract the granule browse image and visualize
Let's look at the browse for this granule

In [72]:
doc = et.fromstring(response.text)
browse_url = doc.findall("result/Granule/AssociatedBrowseImageUrls/ProviderBrowseUrl/URL")[0].text
print("Granule browse URL: " + str(browse_url)) 

img = Image.open(requests.get(browse_url, stream=True).raw)
img. show()

Granule browse URL: https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-public/HLSS30.015/HLS.S30.T18TWL.2021172T153911.v1.5.jpg


New York!

## Where is the data?
We can extract the data locations for this granule as follows:

In [73]:
data_urls = doc.findall("result/Granule/OnlineAccessURLs/OnlineAccessURL/URL")
for url in data_urls:
    print("Granule data URL: " + url.text)

Granule data URL: https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.015/HLS.S30.T18TWL.2021172T153911.v1.5.B02.tif
Granule data URL: https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.015/HLS.S30.T18TWL.2021172T153911.v1.5.B12.tif
Granule data URL: https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.015/HLS.S30.T18TWL.2021172T153911.v1.5.SZA.tif
Granule data URL: https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.015/HLS.S30.T18TWL.2021172T153911.v1.5.B01.tif
Granule data URL: https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.015/HLS.S30.T18TWL.2021172T153911.v1.5.B11.tif
Granule data URL: https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.015/HLS.S30.T18TWL.2021172T153911.v1.5.B8A.tif
Granule data URL: https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.015/HLS.S30.T18TWL.2021172T153911.v1.5.B04.tif
Granule data URL: https://data.lpdaac.earthdatacloud.nasa.gov/

We have a number of data urls. Generally, Sentinel-2 gives you a data file for each band associated with the instrument.

## Obtaining the data
Coming soon!